<a href="https://colab.research.google.com/github/henrim666/ForexHistoricalDataCreator/blob/master/DownloadUnzipCreateDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/gdrive')
!ls -sla /gdrive/'My Drive'/*.*


Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
 16260 -rw------- 1 root root  16650240 Mar 23 01:52 '/gdrive/My Drive/DAT_ASCII_EURCHF_T_201001.db'
 17036 -rw------- 1 root root  17444864 Mar 23 01:52 '/gdrive/My Drive/DAT_ASCII_EURCHF_T_201002.db'
 19228 -rw------- 1 root root  19689472 Mar 23 01:52 '/gdrive/My Drive/DAT_ASCII_EURCHF_T_201003.db'
 21812 -rw------- 1 root root  22335488 Mar 23 01:52 '/gdrive/My Drive/DAT_ASCII_EURCHF_T_201004.db'
 25028 -rw------- 1 root root  25628672 Mar 23 01:52 '/gdrive/My Drive/DAT_ASCII_EURCHF_T_201005.db'
 26148 -rw------- 1 root root  26775552 Mar 23 01:52 '/gdrive/My Drive/DAT_ASCII_EURCHF_T_201006.db'
 24348 -rw------- 1 root root  24932352 Mar 23 01:52 '/gdrive/My Drive/DAT_ASCII_EURCHF_T_201007.db'
 24184 -rw------- 1 root root  24764416 Mar 23 01:52 '/gdrive/My Drive/DAT_ASCII_EURCHF_T_201008.db'
 24044 -rw------- 1 root root  24621056 Mar 23 01:52 '/gdrive/My Drive/DAT_AS

In [0]:
!ls -sla historyforexdata/FX5min
#!mkdir historyforexdata/database

total 8
4 drwxr-xr-x 2 root root 4096 Mar 24 08:31 .
4 drwxr-xr-x 5 root root 4096 Mar 24 08:54 ..


In [0]:
!rm historyforexdata/database/*.*

In [0]:
!ls -sla historyforexdata/FX5min

total 8
4 drwxr-xr-x 2 root root 4096 Mar 24 08:31 .
4 drwxr-xr-x 5 root root 4096 Mar 24 08:31 ..


In [0]:
import csv
import sqlite3
import sqlalchemy
from sqlalchemy import create_engine
from zipfile import ZipFile
import time
from   datetime import timedelta,datetime
import calendar
import pandas as pd
# Implement the default Matplotlib key bindings.
from matplotlib.backend_bases import key_press_handler
from matplotlib.figure import Figure
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
from matplotlib.widgets import TextBox
import os
import logging
import queue
from matplotlib.testing.jpl_units import day
import numpy as np
from scipy import stats
import scipy.optimize
from scipy.optimize import OptimizeWarning
import warnings
import math
from   bs4 import BeautifulSoup
import requests
import csv
import shutil

class TickdataLoader:
  """ Load the tick data from the history databases 
      into a datafeed for plotting  
      startday="20191002",endday="20191011",timeframe="30T",FXpair="USDJPY"
      """
  def __init__(self, startday="20191002",endday="20191011",timeframe="60T",FXpair="USDJPY"):
    self.start                  = startday    
    self.end                    = endday    
    self.steps                  = timeframe    
    self.currencypair           = FXpair 
    self.workpath               = 'historyforexdata' 
    self.databasedir            = '/database/'
    self.databaseprename        = "DAT_ASCII_"
    self.databasetimeframe      = "_T_"
    self.databasefileextension  = ".db"    


  def daterangechart(self,date1, date2):
    """ convert two dates into a range of days """
    for n in range(int ((date2 - date1).days)+1):   
        yield date1 + timedelta(n)

  def gettickdata(self,dailydata,timeframe):
    """ Load the tick data of one day from sql3lite 
        HistoryFX data site loopt 7 uur achter op MT4 dus history + 7 uur
        MT4 loopt 7 uur achter op Tokyo time 
        so HistoryFXday loop 14 hour behind tokyo so +14   
        internal parameters dailydata,timeframe"""
    DBrangeDateSTR  = dailydata.strftime("%Y%m%d")
    DBrangeDate     = datetime.strptime(DBrangeDateSTR, '%Y%m%d')
    DByear          = DBrangeDate.year
    DBmonth         = DBrangeDate.month
    DBday           = DBrangeDate.day
    if DBday<10:      DBdaySTR    = '0' + str(DBday)
    else:             DBdaySTR    = str(DBday) 
    if DBmonth<10:    DBmonthSTR  = '0' + str(DBmonth)
    else:             DBmonthSTR  = str(DBmonth)
    DByearSTR       = str(DByear)
    records = pd.DataFrame(columns=['Symbol','DateTime','Bid','Ask','Tick'])
    databasename = self.currencypair + DByearSTR + DBmonthSTR + "BidAsk"
    dbconnectstring = self.workpath + self.databasedir + self.databaseprename + self.currencypair + self.databasetimeframe + DByearSTR+DBmonthSTR + self.databasefileextension 
    print(dbconnectstring)  
    print(databasename)  
        
    try:      
        conn                = sqlite3.connect(dbconnectstring)
        cur                 = conn.cursor()
        selecttablestring   = 'SELECT  Symbol,DateTime,Bid,Ask,Tick FROM (SELECT  Symbol,DateTime,Bid,Ask,Tick,COUNT(*) c   FROM ' + databasename + ' WHERE DateTime LIKE ' + ' \047' + DByearSTR+DBmonthSTR+DBdaySTR + '%\047 GROUP BY DateTime HAVING c=1)'
        records             = pd.read_sql_query(selecttablestring, conn)
        records['DateTime'] = pd.to_datetime(records['DateTime'], format='%Y%m%d %H%M%S%f') + timedelta(hours=14) 
        return(records)
    except:
        print("Check if GDrive is mounted")
        return()

  def getplotdata(self,startday="20191002",endday="20191011",timeframe="60T"):
    """ Load the tick data of one day from sql3lite 
        HistoryFX data site loopt 7 uur achter op MT4 dus history + 7 uur
        MT4 loopt 7 uur achter op Tokyo time 
        so HistoryFXday loop 14 hour behind tokyo so +14   
        startday="20191002",endday="20191011",timeframe="30T" """
    DBstartDate = datetime.strptime(startday, '%Y%m%d')
    DBstartDate = DBstartDate - timedelta(days=1)# reduce startdate with one day to get the start data of the startday -14 hours shift
    DBendDate   = datetime.strptime(endday , '%Y%m%d')
    DBendDate   = DBendDate 
    DataRangeTimeFrame = pd.DataFrame()
    for dailydata in self.daterangechart(DBstartDate, DBendDate):
      print(dailydata)
      DataRangeTimeFrame = DataRangeTimeFrame.append(self.gettickdata(dailydata,timeframe)).drop_duplicates( keep='last') 

    print('finished appending data')
    DataRangeTimeFrame = DataRangeTimeFrame.set_index(pd.DatetimeIndex(DataRangeTimeFrame['DateTime']))
    print('finished index')
    DataRangeTimeFrame['Bid']   = DataRangeTimeFrame['Bid'].astype('float') 
    DataRangeTimeFrame['Ask']   = DataRangeTimeFrame['Ask'].astype('float') 
    DataRangeTimeFrame['Tick']  = DataRangeTimeFrame['Tick'].astype('int') 
    DataRangeTimeFrame['Tick']  = 1
    #print(DataRangeTimeFrame)
    VolumeTickdata=DataRangeTimeFrame.resample(timeframe).sum().copy(deep=True)
    MeanTickdata=DataRangeTimeFrame.resample(timeframe).mean().copy(deep=True)
    OpenTickdata=DataRangeTimeFrame.resample(timeframe).bfill().copy(deep=True)
    LowTickdata=DataRangeTimeFrame.resample(timeframe).min().copy(deep=True)
    HighTickdata=DataRangeTimeFrame.resample(timeframe).max().copy(deep=True)
    Framedata=DataRangeTimeFrame.resample(timeframe).bfill().copy(deep=False)
    Framedata.drop('Tick',  axis=1, inplace=True)
    Framedata.drop('Symbol',axis=1, inplace=True)
    Framedata.drop('Ask',   axis=1, inplace=True)
    Framedata.drop('Bid',   axis=1, inplace=True)
    Framedata["DateTime"] = Framedata["DateTime"].apply(mdates.date2num)
    Framedata['Open']     = OpenTickdata['Bid'] 
    Framedata['High']     = HighTickdata['Bid'] 
    Framedata['Low']      = LowTickdata['Bid'] 
    Framedata['Close']    = Framedata['Open'].shift(-1)
    Framedata['Volume']   = VolumeTickdata['Tick'] 
    Framedata['Mean']     = MeanTickdata['Bid'] 
    Framedata             = Framedata[:-1] 
    print('finished column creation')
    return(Framedata)  

def get_FXdata(FXpairs,dates):
  df             = pd.DataFrame(index=dates)
  DBstart_date   = str(df.head(1).index.values)[:12][2:].replace("-","")
  DBend_date     = str(df.tail(1).index.values)[:12][2:].replace("-","")
  #if 'USDJPY' not in FXpairs:  # add USDJPY for reference, if absent
  #  FXpairs.insert(0, 'USDJPY')
  #df_initial     = TickdataLoader(startday=DBstart_date,endday=DBend_date,timeframe=timeframe,FXpair=FXpairs[0]).getplotdata(startday=DBstart_date,endday=DBend_date,timeframe=timeframe)
  #df             = df.join(df_initial,how="inner")
  #print(df)
  for FXpair in FXpairs:
    print(FXpair)
    #load candles
    df_temp = TickdataLoader(startday=DBstart_date,endday=DBend_date,timeframe=timeframe,FXpair=FXpair).getplotdata(startday=DBstart_date,endday=DBend_date,timeframe=timeframe)
    print('loaded from db ',df_temp)
    # iterating the columns 
    for col in df_temp.columns: 
      print(col , (FXpair+col))
      df_temp = df_temp.rename(columns={col:(FXpair+col)})
    df = df.join(df_temp) # default left join 
    print(df) 

  return(df)  

def downloadzipfile(zipfiletype,zipfiletimeframe,zipfilefxpair,zipfileyear,zipfilemonth):
    postuseragent     = 'Mozilla/5.1'
    postorigin        = 'http://www.histdata.com'
    posturl           = postorigin+'/download-free-forex-historical-data/?/'+zipfiletype+'/'+zipfiletimeframe+'/'+zipfilefxpair+'/'+zipfileyear+'/'+zipfilemonth
    targetfolder      = 'historyforexdata/'
    #HISTDATA_COM_MT_EURUSD_M1_2018.zip
    print("downloadzipfile" , posturl)
    # get the page and make the soup
    r = requests.get(posturl)
    data = r.text
    soup = BeautifulSoup(data, "lxml")
    #div style="display:none;"
    table = soup.find("div", style="display:none;")
    #print(table)
    try:
        posttk = table.find('input', {'id': 'tk'}).get('value')
        print(posttk)
    except:
        pass    
    try:
        postdate = table.find('input', {'id': 'date'}).get('value')
        print(postdate)
    except:
        pass    
    try:
        postdatemonth = table.find('input', {'id': 'datemonth'}).get('value')
        print(postdatemonth)
    except:
        pass    
    try:
        postplatform = table.find('input', {'id': 'platform'}).get('value')
        print(postplatform)
    except:
        pass   
    try:
        posttimeframe = table.find('input', {'id': 'timeframe'}).get('value')
        print(posttimeframe)
    except:
        pass    
    try:
        postfxpair = table.find('input', {'id': 'fxpair'}).get('value')
        print(postfxpair)
    except:
        pass
    
    #if posttimeframe == "M1":
    #    posttimeframe = posttimeframe+"_"
    targetfilename    ='HISTDATA_COM_'+postplatform+'_'+postfxpair+'_'+posttimeframe+'_'+postdatemonth+'.zip'
    targetpathfilename=targetfolder+targetfilename
    print("targetfilename",targetfilename, posttimeframe )
    print("targetpathfilename",targetpathfilename)
    
    resp    = requests.post(postorigin+'/get.php', 
    data    = {'tk': posttk, 'date': postdate, 'datemonth': postdatemonth, 'platform': postplatform, 'timeframe': posttimeframe, 'fxpair': postfxpair},
    headers = {'User-Agent': postuseragent, 'Origin': postorigin, 'Referer': posturl})

    # wait here for the file to download
    result = None
    while result is None:
        with open(targetpathfilename, 'wb') as fpw:
            for chunk in resp.iter_content():
                fpw.write(chunk)
        time.sleep(1)
        result = 1

def getFXrawTickdata(FXpair,startY,endY):
    timeframe = '_T_'
    sourcefileextension = '.csv'  
    targetfileextension = '.db'
    sourceprename = 'DAT_ASCII_'
    targetprename = 'DAT_ASCII_'
    workpath = 'historyforexdata/'
    sourcedir = 'unzip/'
    targetdir = 'database/'
    print('Extract all ZIPfiles from history fx  ')
    symbolsub = [FXpair]
    for symbolsubstring in symbolsub:
        for yearsub in range (startY, (int(endY)+1)):
            for monthsub in range(1, 13):
                filetype = 'ascii'
                filetimeframe = 'tick-data-quotes'
                currencypair = symbolsubstring
                fileyear = str(yearsub)
                filemonth = str(monthsub)
                print(filetype,filetimeframe,currencypair,fileyear,filemonth)
                downloadzipfile(filetype,filetimeframe,currencypair,fileyear,filemonth)
                if monthsub<10:
                    monthsubstring = '0' + str(monthsub)
                else: 
                    monthsubstring = str(monthsub) 
                zipfilename='historyforexdata/HISTDATA_COM_ASCII_' + symbolsubstring + '_T_' + str(yearsub)+monthsubstring + '.zip'
                print(zipfilename)
                try:
                    with ZipFile(zipfilename, 'r') as zipObj:
                        zipObj.extractall('historyforexdata/unzip/')
                except OSError:
                    print("error ")
                    pass
                os.remove(zipfilename)
                dbconnectstring =    workpath + targetdir + targetprename +  symbolsubstring + timeframe + str(yearsub)+monthsubstring + targetfileextension     
                print(dbconnectstring)  
                conn = sqlite3.connect(dbconnectstring)
                cur = conn.cursor()
                databasename = symbolsubstring +str(yearsub)+monthsubstring + "BidAsk"
                createtablestring  = 'CREATE TABLE '  + databasename + ' (Symbol text, DateTime text, Bid text, Ask text, Tick text)'
                print(createtablestring)
                cur.execute(createtablestring)
                pandaconnectstring  =  workpath + sourcedir + sourceprename + symbolsubstring + timeframe + str(yearsub)+monthsubstring + sourcefileextension 
                try:
                    print('source csv file for panda ',pandaconnectstring)
                    print('target database file ',dbconnectstring)
                    with open(pandaconnectstring, "r") as f: # CSV file input
                        reader = csv.reader(f, delimiter=',') # no header information with delimiter
                        for row in reader:
                            Symbol   = symbolsubstring
                            DateTime = str(row[0])
                            Bid      = str(row[1])
                            Ask      = str(row[2])
                            Tick     = str(row[3])
                            sql = "INSERT INTO " + databasename + " VALUES " + "('" + Symbol + "','"  + DateTime  + "','"  + Bid  + "','"  + Ask+ "','"  + Tick + "')"
                            cur.execute(sql)
                except OSError:
                    print("error ")
                    pass
                conn.commit()
                os.remove(pandaconnectstring)
                os.remove(workpath + sourcedir + sourceprename + symbolsubstring + timeframe + str(yearsub)+monthsubstring +".txt") 
    conn.close()

    
    
#os.makedirs("historyforexdata", exist_ok=True)
#s.makedirs("historyforexdata/unzip", exist_ok=True)
#os.makedirs("historyforexdata/database", exist_ok=True)
#os.makedirs("historyforexdata/FX5min", exist_ok=True)

#getFXrawTickdata("NZDCAD",2019,2019)  

# add transfor in to 5 min and all technical data 

In [0]:
os.makedirs("historyforexdata", exist_ok=True)
shutil.rmtree("historyforexdata/unzip")
os.makedirs("historyforexdata/unzip", exist_ok=True)
shutil.rmtree("historyforexdata/database")
os.makedirs("historyforexdata/database", exist_ok=True)
os.makedirs("historyforexdata/FX5min", exist_ok=True)

for years in range(2012,2020):
  shutil.rmtree("historyforexdata/database")
  os.makedirs("historyforexdata/database", exist_ok=True)
  startyearint=years
  endnextyearint=startyearint+1
  startyear=str(startyearint)
  endnextyear=str(endnextyearint)
  start_date     = startyear+"-01-02 00:00:00" ; end_date   = startyear+"-12-31 23:55:00" #dates 
  timeframe      = "5T"
  dates          = pd.date_range(start_date, end_date, freq=timeframe) # Define a date range
  rawdata_columns= ["Symbol","DateTime","Bid","Ask","Tick"]
  #FXpairs        = ["USDJPY","EURJPY","GBPJPY","GBPUSD","EURGBP","BCOUSD","EURUSD","JPXJPY","NSXUSD","SPXUSD","UDXUSD","UKXGBP","XAGUSD","XAUGBP"]
  FXpairs        = ["NZDCAD"]
  getFXrawTickdata("NZDCAD",years,years)
  df201910=get_FXdata(FXpairs,dates)
  print(df201910)


  # Database connector
  engine_type             = "sqlite:///"
  databasedir             = '/gdrive/My Drive/'
  databasename            = "FX"+startyear+"NZDCADtest"
  databasefileextension   = ".db"
  engineSTR=engine_type+databasedir+databasename+databasefileextension
  print(engineSTR)
  engine = create_engine(engineSTR)

  sql="SELECT * FROM \047FX"+startyear+"NZDCADtest\047"

  df201910.to_sql(databasename, con=engine, if_exists='replace')
  print("Data loaded")
  df2 = pd.read_sql_query(sql, engine)
  print(df2)
      
  #INSERT INTO table2 SELECT DISTINCT * FROM table1

Streaming output truncated to the last 5000 lines.
CREATE TABLE NZDCAD201211BidAsk (Symbol text, DateTime text, Bid text, Ask text, Tick text)
source csv file for panda  historyforexdata/unzip/DAT_ASCII_NZDCAD_T_201211.csv
target database file  historyforexdata/database/DAT_ASCII_NZDCAD_T_201211.db
ascii tick-data-quotes NZDCAD 2012 12
downloadzipfile http://www.histdata.com/download-free-forex-historical-data/?/ascii/tick-data-quotes/NZDCAD/2012/12
61ea856801ef518541df06bf6d87c981
2012
201212
ASCII
T
NZDCAD
targetfilename HISTDATA_COM_ASCII_NZDCAD_T_201212.zip T
targetpathfilename historyforexdata/HISTDATA_COM_ASCII_NZDCAD_T_201212.zip
historyforexdata/HISTDATA_COM_ASCII_NZDCAD_T_201212.zip
historyforexdata/database/DAT_ASCII_NZDCAD_T_201212.db
CREATE TABLE NZDCAD201212BidAsk (Symbol text, DateTime text, Bid text, Ask text, Tick text)
source csv file for panda  historyforexdata/unzip/DAT_ASCII_NZDCAD_T_201212.csv
target database file  historyforexdata/database/DAT_ASCII_NZDCAD_T_20121

In [0]:
from multiprocessing import Process, Queue
import numpy as np
import pandas as pd
import datetime as dt
from pandas_datareader import DataReader
import logging
import sqlite3
import sqlalchemy
from sqlalchemy import create_engine
import statistics
import math

#max leverage 25
#margine 2%
#balance * Close * 0.02 = leverage 
#(balance - leverage)*50 = tradable balance 
#tradable balance / 100 or 1000 or 10000 or 100000
#https://thismatter.com/money/forex/leverage-margin-pips.htm 
class Portfolio:
    def __init__(self, balance=1000000): 
        self._portfolio = {} #dict ?
        self._portfolio['**CASH**'] = {'Shares' : balance, 'Price' : 1.0, 'Updates' : 1}
        print("init portfollo : ",self._portfolio)
        # Database connector
        self._engine_type             = "sqlite:///"
        self._databasedir             = 'database/'
        self._databasename            = "Portfolio"
        self._databasefileextension   = ".db"
        self._engineSTR=self._engine_type+self._databasedir+self._databasename+self._databasefileextension
        self._engine = create_engine(self._engineSTR)

    def update(self, price, ticker, timestamp):
        pfdf = pd.DataFrame(columns=['timestamp', 'ticker', 'Price', 'Updates', 'shares'])
        if ticker in self._portfolio:
            self._portfolio[ticker]['Price'] = price
            self._portfolio[ticker]['Updates'] = self._portfolio[ticker]['Updates'] + 1
            pfdf = pfdf.append({'timestamp': timestamp, 'ticker': ticker, 'Price': price, 'Updates': self._portfolio[ticker]['Updates'], 'shares': self._portfolio[ticker]['Shares']}, ignore_index=True)
        else:
            self._portfolio[ticker] = {}
            self._portfolio[ticker]['Price'] = price
            self._portfolio[ticker]['Shares'] = 0
            self._portfolio[ticker]['Updates'] = 1
            pfdf = pfdf.append({'timestamp': timestamp, 'ticker': ticker, 'Price': price, 'Updates': self._portfolio[ticker]['Updates'], 'shares': self._portfolio[ticker]['Shares']}, ignore_index=True)
        pfdf.to_sql(self._databasename, con=self._engine, if_exists='append')

    @property
    def balance(self):
        return self._portfolio['**CASH**']['Shares']

    @balance.setter
    def balance(self, balance):
        self._portfolio['**CASH**']['Shares'] = balance

    def adjust_balance(self, delta):
        self._portfolio['**CASH**']['Shares'] = self.balance + delta

    def __contains__(self, item):
        return (item in self._portfolio)

    def value_summary(self, date):
        sum = self.get_total_value()
        return '%s : Stock value: %s, Cash: %s, Total %s' % (date, sum-self.balance, self.balance, sum)

    def get_total_value(self):
        sum = 0
        for stock in self._portfolio.values():
            sum += stock['Shares'] * stock['Price']
        return sum

    def get_value(self, ticker):
        return self.get_shares(ticker) * self.get_price(ticker)

    def get_price(self, ticker):
        return self._portfolio[ticker]['Price']

    def get_shares(self, ticker):
        return self._portfolio[ticker]['Shares']

    def get_update_count(self, ticker):
        return self._portfolio[ticker]['Updates']

    def set_shares(self, ticker, shares):
        self._portfolio[ticker]['Shares'] = shares

    def update_shares(self, ticker, share_delta):
        self.set_shares(ticker, self.get_shares(ticker) + share_delta)

    def update_trade(self, ticker, share_delta, price, fee):
        # Assumes negative shares are sells, requires validation from Controller
        self.set_shares(ticker, self.get_shares(ticker) + share_delta)
        self.adjust_balance(-(price*share_delta + fee))

    def __str__(self):
        return self._portfolio.__str__()


class Algorithm:
    '''
    Example algorithm for trading. Must implement a "generate_orders" function which returns a list of orders.
    Each order is a tuple of the form
        ( Stock Ticker str, Current Price float, Order Amount in shares float)
    Algorithm trades for stocks via a rolling window strategy, and randomly liquidates positions
    '''
    def __init__(self):
        self._averages   = {}
        self._averages1  = {}
        self._averages2  = {}
        self._averages10 = {}
        self._lambda = .5
        self._fee_estimate = lambda x : x*.04+10
        self._updates = 0
        self._price_window   = 4    #20 day avarage 
        self._price_window1  = 156  #200 day avarage 
        self._price_window2  = 16   #52 day avarage 
        self._price_window10 = 310  #310 day avarage 
        self._trend   = np.zeros(self._price_window)
        self._trend1  = np.zeros(self._price_window1)
        self._trend2  = np.zeros(self._price_window2)
        self._trend10 = np.zeros(self._price_window10)
        self._minimum_wait_between_trades = 15 # Must be less than price window
        self._last_trade = 0
        self._last_date = None        
        # Database connector
        self._engine_type             = "sqlite:///"
        self._databasedir             = 'database/'        
        self._databasename            = "Algorithm"
        self._databasefileextension   = ".db"
        self._engineSTR=self._engine_type+self._databasedir+self._databasename+self._databasefileextension
        print(self._engineSTR)
        self._engine = create_engine(self._engineSTR)
        self._DFOneDayHistory=pd.DataFrame()

    def add_stock(self, stock, price):
        self._averages[stock] = price

    def _determine_if_trading(self, date, portfolio_value, cash_balance):
        from datetime import datetime
        time_delay_met = True
        trade = False
        override = False
        self._updates += 1
        datetimeFormat = '%Y-%m-%d %H:%M:%S'
        #can this be an issue (dt - last_dt) if there is a weekend in between ?? YES  dtdiff 2 days, 0:10:00 invalid literal for int() with base 10: '2 days, 0'
        if self._last_date is not None:
            dt           = datetime.strptime(date[:-7], datetimeFormat)
            last_dt      = datetime.strptime(self._last_date[:-7], datetimeFormat)
            dtdiff       = (dt - last_dt)    
            hour,min,sec = str(dtdiff).split(":")
            #hour=int(hour);;sec=int(sec)
            min=int(min)
            if min <= self._minimum_wait_between_trades:
                return False
        if self._updates == self._price_window+1:
            trade = True
        if (np.mean(self._trend)-portfolio_value)/portfolio_value > 0.05:
            override = True
        if cash_balance > portfolio_value*.03:
            override = True
        return trade or override

    def generate_orders(self, timestamp, portfolio):
        orders          = []
        cash_balance    = portfolio.balance
        portfolio_value = portfolio.get_total_value()
        self.add_trend_value(portfolio_value)
        #check if money is ok to trade
        if not self._determine_if_trading(timestamp,portfolio_value,cash_balance):
            return orders
        #check if the price window 1 is the biggest is filled for stock/FXpair if not return     
        valid_stocks = [stock for stock in self._averages if portfolio.get_update_count(stock) > self._price_window1]
        if len(valid_stocks) == 0:
            return orders
        #real trading loop
        for stock in np.random.choice(valid_stocks, replace=False, size=len(valid_stocks)):
            amt               = (cash_balance / len(valid_stocks))*0.05 # Spend available cash only 5percent
            relative_change   = (self.get_window_average(stock=stock)  - self.get_price(stock)) /self.get_price(stock)
            relative_change1  = (self.get_window1_average(stock=stock) - self.get_price1(stock))/self.get_price1(stock)
            relative_change2  = (self.get_window2_average(stock=stock) - self.get_price2(stock))/self.get_price2(stock)
            relative_change10 = (self.get_window10_average(stock=stock)- self.get_price10(stock))/self.get_price10(stock)
            if abs(relative_change) > 0.0012:
                # Positive is buy, negative is sell
                order_type = np.sign(relative_change)
                if order_type > 0 and np.random.uniform(0,1,size=1)[0] < .9:
                    amt = np.round(amt/self.get_price(stock),-2)#round up into 100 
                else:
                    amt = - portfolio.get_shares(stock) # Liquidate! Why not?
                if abs(amt) < .01:
                    # Stop small trades
                    continue
                orders.append((stock, self.get_price(stock), amt))
        self._last_trade = self._updates
        self._last_date = timestamp

        return orders

    def get_window_average(self, stock):
        return np.mean(self._averages[stock]['History'])

    def get_window_standard_deviation(self, stock):
        return np.std(self._averages[stock]['History'])

    def get_window_min(self, stock):
        return np.min(self._averages[stock]['History'])

    def get_window_max(self, stock):
        return np.max(self._averages[stock]['History'])


    def get_window1_average(self, stock):
        return np.mean(self._averages1[stock]['History1'])

    def get_window1_standard_deviation(self, stock):
        return np.std(self._averages1[stock]['History1'])

    def get_window1_min(self, stock):
        return np.min(self._averages1[stock]['History1'])

    def get_window1_max(self, stock):
        return np.max(self._averages1[stock]['History1'])

    def get_window2_average(self, stock):
        return np.mean(self._averages2[stock]['History2'])

    def get_window2_standard_deviation(self, stock):
        return np.std(self._averages2[stock]['History2'])

    def get_window2_min(self, stock):
        return np.min(self._averages2[stock]['History2'])

    def get_window2_max(self, stock):
        return np.max(self._averages2[stock]['History2'])

    def get_window10_average(self, stock):
        return np.mean(self._averages10[stock]['History10'])

    def get_window10_standard_deviation(self, stock):
        return np.std(self._averages10[stock]['History10'])

    def get_window10_min(self, stock):
        return np.min(self._averages10[stock]['History10'])

    def get_window10_max(self, stock):
        return np.max(self._averages10[stock]['History10'])

    def get_window10_DataFrame(self, stock , timestamp):
        #does this work ?
        return self._averages10[stock]['History10']


    def update(self, stock, price, timestamp):
        if stock in self._averages:
            self.add_price(stock, price)
        else:
            length = self._price_window
            self._averages[stock] = {'History' : np.zeros(length), 'Index' : 0, 'Length' : length}
            data = self._averages[stock]['History']
            data[0] = price
        #adf=pd.DataFrame.from_dict(self._averages)
        #adf["timestamp"]=timestamp
        #pd.options.display.max_rows = 999
        #print(adf)
        #adf.to_sql(self._databasename, con=self._engine, if_exists='append')
        #print("\n Algorithm : ",timestamp,price,stock,self._averages[stock])    

    def update1(self, stock, price, timestamp):
        if stock in self._averages1:
            self.add_price1(stock, price)
        else:
            length = self._price_window1
            self._averages1[stock] = {'History1' : np.zeros(length), 'Index1' : 0, 'Length1' : length}
            data = self._averages1[stock]['History1']
            data[0] = price
        #adf=pd.DataFrame.from_dict(self._averages)
        #adf["timestamp"]=timestamp
        #pd.options.display.max_rows = 999
        #print(adf)
        #adf.to_sql(self._databasename, con=self._engine, if_exists='append')
        #print("\n Algorithm1 : ",timestamp,price,stock,self._averages1[stock]) 

    def update2(self, stock, price, timestamp):
        if stock in self._averages2:
            self.add_price2(stock, price)
        else:
            length = self._price_window2
            self._averages2[stock] = {'History2' : np.zeros(length), 'Index2' : 0, 'Length2' : length}
            data = self._averages2[stock]['History2']
            data[0] = price
        #adf=pd.DataFrame.from_dict(self._averages)
        #adf["timestamp"]=timestamp
        #pd.options.display.max_rows = 999
        #print(adf)
        #adf.to_sql(self._databasename, con=self._engine, if_exists='append')
        #print("\n Algorithm2 : ",timestamp,price,stock,self._averages2[stock]) 

    def update10(self, stock, price, timestamp):
        if stock in self._averages10:
            self.add_price10(stock, price)
        else:
            length = self._price_window10
            self._averages10[stock] = {'History10' : np.zeros(length), 'Index10' : 0, 'Length10' : length}
            data = self._averages10[stock]['History10']
            data[0] = price
        #adf=pd.DataFrame.from_dict(self._averages)
        #adf["timestamp"]=timestamp
        #pd.options.display.max_rows = 999
        #print(adf)
        #adf.to_sql(self._databasename, con=self._engine, if_exists='append')
        #print("\n Algorithm2 : ",timestamp,price,stock,self._averages2[stock]) 


    def get_price(self, stock):
        # Assumes history is full
        return self._averages[stock]['History'][-1]

    def get_price1(self, stock):
        # Assumes history1 is full
        return self._averages1[stock]['History1'][-1]

    def get_price2(self, stock):
        # Assumes history2 is full
        return self._averages2[stock]['History2'][-1]

    def get_price10(self, stock):
        # Assumes history10 is full
        return self._averages10[stock]['History10'][-1]


    def add_price(self, stock, price):
        history = self._averages[stock]['History']
        ind = self._averages[stock]['Index']
        length = self._averages[stock]['Length']
        if ind < length-1:
            history[ind+1] = price
            self._averages[stock]['Index'] = ind + 1
        elif ind == length-1:
            history[:-1] = history[1:]
            history[-1] = price

    def add_price1(self, stock, price):
        history = self._averages1[stock]['History1']
        ind = self._averages1[stock]['Index1']
        length = self._averages1[stock]['Length1']
        if ind < length-1:
            history[ind+1] = price
            self._averages1[stock]['Index1'] = ind + 1
        elif ind == length-1:
            history[:-1] = history[1:]
            history[-1] = price

    def add_price2(self, stock, price):
        history = self._averages2[stock]['History2']
        ind = self._averages2[stock]['Index2']
        length = self._averages2[stock]['Length2']
        if ind < length-1:
            history[ind+1] = price
            self._averages2[stock]['Index2'] = ind + 1
        elif ind == length-1:
            history[:-1] = history[1:]
            history[-1] = price

    def add_price10(self, stock, price):
        history = self._averages10[stock]['History10']
        ind     = self._averages10[stock]['Index10']
        length  = self._averages10[stock]['Length10']
        if ind < length-1:
            history[ind+1] = price
            self._averages10[stock]['Index10'] = ind + 1
        elif ind == length-1:
            history[:-1] = history[1:]
            history[-1]  = price


    def add_trend_value(self, value):
        history = self._trend
        if self._updates <= self._price_window - 1:
            history[self._updates] = value
        elif self._updates > self._price_window-1:
            history[:-1] = history[1:]
            history[-1] = value 

    def add_trend1_value(self, value):
        history = self._trend1
        if self._updates <= self._price_window1 - 1:
            history[self._updates] = value
        elif self._updates > self._price_window1-1:
            history[:-1] = history[1:]
            history[-1] = value 

    def add_trend2_value(self, value):
        history = self._trend2
        if self._updates <= self._price_window2 - 1:
            history[self._updates] = value
        elif self._updates > self._price_window2-1:
            history[:-1] = history[1:]
            history[-1] = value 

    def add_trend10_value(self, value):
        history = self._trend10
        if self._updates <=  self._price_window10-1:
            history[self._updates] = value
        elif self._updates > self._price_window10-1:
            history[:-1] = history[1:]
            history[-1]  = value 

class OrderApi:
    def __init__(self):
        #self._slippage_std = .01
        self._slippage_std = 0 #make all fee free
        self._prob_of_failure = .0001
        #self._fee = .02       #make all fee free
        #self._fixed_fee = 10  #make all fee free
        self._fee = 0
        self._fixed_fee = 0
        self._calculate_fee = lambda x : self._fee*abs(x) + self._fixed_fee

    def process_order(self, order):
        slippage = np.random.normal(0, self._slippage_std, size=1)[0]
        if np.random.choice([False, True], p=[self._prob_of_failure, 1 -self._prob_of_failure],size=1)[0]:
            trade_fee = self._fee*order[1]*(1+slippage)*order[2]
            return (order[0], order[1]*(1+slippage), order[2], self._calculate_fee(trade_fee))


class DataSource:
    '''
    Data source for the backtester. Must implement a "get_data" function
    which streams data from the data source.
    '''
    def __init__(self, source='yahoo', tickers=['GOGL','AAPL'], start = dt.datetime(2020,1,1), end=dt.datetime.today()):
        self.set_source_stock(source = source, tickers= tickers, start=start, end=end)

    @classmethod
    def process(cls, queue, source = None):
            source = cls() if source is None else source
            while True:
                data = source.get_data()
                if data is not None:
                    queue.put(data)
                    if data == 'POISON':
                        break

    def getFXpairdata(self, FXpairs):
        startdate = "2018-01-01"
        enddate   = "2018-12-31"
        conn      = sqlite3.connect("/content/sample_data/FX2018all.db")
        c         = conn.cursor()
        c.execute("select name FROM PRAGMA_TABLE_INFO('FX2018all') WHERE name like'"+"FXnews"+"%' ")
        newsrow   = [item[0] for item in c.fetchall()]
        pairrow=[]
        for FXpair in FXpairs:
          c.execute("select name FROM PRAGMA_TABLE_INFO('FX2018all') WHERE name like'"+FXpair+"%' ")
          fxrow   = [item[0] for item in c.fetchall()]
          pairrow = pairrow + fxrow
        row = pairrow + newsrow
        newnewsclmn = []
        newnewsclmn.append("DateTime")
        for cell in newsrow:
            cell    = cell[6:]
            newnewsclmn.append(cell)
        command = """select DISTINCT "index" , """+ " , ".join(row) +""" FROM FX2018all WHERE ("index" BETWEEN """ +"\047"+startdate+"%\047" + " AND " + "\047"+enddate+"%\047 ) AND "+FXpairs[0]+"DateTime > 1 " + """ ORDER BY "index"  """
        print(command)
        df          = pd.DataFrame(pd.read_sql_query(command ,conn))
        df.columns  = [col.replace("FXnews", '') for col in df.columns]
        return(df)

    def set_source_stock(self, source, tickers, start, end):
        FXpairs=tickers
        FXCandles= self.getFXpairdata(FXpairs)
        events = []
        for index,row in FXCandles.iterrows():
            for FXpair in FXpairs:
              events.append((row['index'],FXpair,float(row[FXpair+"DateTime"]),float(row[FXpair+"Open"]),float(row[FXpair+"Close"]),float(row[FXpair+"High"]),float(row[FXpair+"Low"])))
        self._source = events


    def get_data(self):
        try:
            #print(self._source.pop(0))
            return self._source.pop(0)
        except IndexError as e:
            return 'POISON'

class Controller:
    def __init__(self, portfolio = None, algorithm = None):
        self._portfolio = Portfolio() if portfolio is None else portfolio
        self._algorithm = Algorithm() if algorithm is None else algorithm
        self._order_api = OrderApi()

    #https://www.youtube.com/watch?v=rq8cL2XMM5M
    @classmethod
    def backtest(cls, queue, controller = None):
        controller              = cls() if controller is None else controller
        engine_type             = "sqlite:///"
        databasedir             = 'database/'
        databasename            = "TradeOrder"
        databasefileextension   = ".db"
        engineSTR               = engine_type+databasedir+databasename+databasefileextension
        engine                  = create_engine(engineSTR)
        TimeSTARTTokyo   = "09:00:00.000000"
        TimeENDTokyo     = "10:00:00.000000"
        TimeSTARTLondon  = "17:00:00.000000"
        TimeENDLondon    = "18:00:00.000000"
        TimeSTARTNewYork = "22:00:00.000000"
        TimeENDNewYork   = "23:00:00.000000"
        OpenMarkets      = ""
        oldcross16ma4    = 999
        oldcross16ma156  = 999
        oldcross4ma156   = 999        
        oldcross16std4   = 999
        oldcross16std156 = 999
        oldcross4std156  = 999
        oldcross048mvg006= 999
        oldcross048mvg288= 999
        oldcross006mvg288= 999         
        oldcross052mvg020= 999
        oldcross052mvg200= 999
        oldcross020mvg200= 999 
        delaytrade       = 8;previoustrade    = 0
        value            = 0;trade            = 0
        tradeprofit      = 0;tradecut         = 0
        tradelength      = 0;MACD             = 0
        MAVG288     = 0;MAVG48     = 0;MAVG6      = 0
        MAVG200tech = 0;MAVG52tech = 0;MAVG20tech = 0
        STD200tech  = 0;STD52tech  = 0;STD20tech  = 0        
        exp1             = 0
        exp2             = 0;exp3             = 0
        RSI1             = 0;RSI2             = 0
        r3=value;r2=value;r1=value
        s3=value;s2=value;s1=value
        RSItradeOrder=""; RSIhighOrder=""; RSIhigh=0; RSItrade=0; MACDhigh=0; MACDtrade=0
        commandpullhistory="""SELECT * FROM TradeOrder WHERE Datetime > 1 AND high > 1 ORDER BY 'index' desc"""
        # below senstivity levels look to work fine
        colorline= 'lightgrey'
        sensitivelvlRSI=.3
        sensitivelvlMACD=.05 
        RSI1high=0;RSI2high=0;MACDhigh=0;RSI1highOrder="";RSI2highOrder="";RSItradeOrder=""
        candlecountangle=-1
        angle=9999;innerhigh=-9999;innerlow=9999;maxhighdistance=-9999;maxlowdistance=9999;surfaceabove=-9999;candletotalcount=0 
        previousclose=0
        previoustradetime=0
        totaldiff=0
        #tradeslicedf=pd.DataFrame()
        #tradeslicedf=pd.DataFrame(columns=['timestamp','close','angle','delta','candlescount'])
        try:
            while True:
                if not queue.empty():
                    o = queue.get()
                    #print(o)
                    if o == 'POISON':
                        print("POISON")
                        break
                    timestamp = o[0]
                    ticker    = o[1]
                    DateTime  = o[2]
                    open      = o[3]
                    close     = o[4]
                    high      = o[5]
                    low       = o[6]

                    candlecountangle=candlecountangle + 1

                    # Update pricing
                    controller.process_pricing(ticker = ticker, price = close, timestamp = timestamp)
                    # Generate Orders look if the alg sees a trading oppertunity
                    orders = controller._algorithm.generate_orders(timestamp, controller._portfolio) 

                    maprice4   = controller._algorithm.get_window_average(ticker) 
                    maprice156 = controller._algorithm.get_window1_average(ticker) 
                    maprice16  = controller._algorithm.get_window2_average(ticker) 

                    stdprice4   = controller._algorithm.get_window_standard_deviation(ticker) 
                    stdprice156 = controller._algorithm.get_window1_standard_deviation(ticker) 
                    stdprice16  = controller._algorithm.get_window2_standard_deviation(ticker) 

                    minprice4    = controller._algorithm.get_window_min(ticker) 
                    minprice156  = controller._algorithm.get_window1_min(ticker) 
                    minprice16   = controller._algorithm.get_window2_min(ticker) 
                    maxprice4    = controller._algorithm.get_window_max(ticker) 
                    maxprice156  = controller._algorithm.get_window1_max(ticker) 
                    maxprice16   = controller._algorithm.get_window2_max(ticker) 

                    column_names = ["close"]
                    dfonedY      = pd.DataFrame(controller._algorithm.get_window10_DataFrame(ticker,timestamp),columns = column_names)
                    dfonedY      = dfonedY.replace(0, np.nan)
                    dfonedY      = dfonedY.dropna(how='all', axis=0) 
                    #print(dfonedY,len(dfonedY)) 
                    if len(dfonedY)>301:
                      #DFMAVG288 = dfonedY.rolling(window=288).mean()
                      #DFlastMAVG288row=DFMAVG288.tail(1)
                      #print(DFlastMAVG288row)
                      #print(float(DFlastMAVG288row["close"].values))
                      #start MACD RSI
                      # we keep the default time sample windows for MACD and RSI calculations 
                      # MACD 12 - 26 and not used is the 9 
                      # 1 hour - 2 1/8 hours and let is the 45 min 
                      # RSI used 12 so a 1 hour sample window  
                      # 200 52 20 add these std and movavg 
                      dfRM = dfonedY[['close']]
                      dfRM.dropna(subset=['close'])
                      delta = dfRM.diff()
                      delta = delta[1:] 
                      # Make the positive gains (up) and negative gains (down) Series
                      up, down = delta.copy(), delta.copy()
                      up[up < 0] = 0
                      down[down > 0] = 0
                      window_length = 12
                      DFRSI1 = ((((up.ewm(span=window_length).mean())/(down.abs().ewm(span=window_length).mean()))))
                      DFRSI2 = ((((up.rolling(window_length).mean())/(down.abs().rolling(window_length).mean()))))
                      DFexp1 = dfRM.ewm(span=12, adjust=False).mean()
                      DFexp2 = dfRM.ewm(span=26, adjust=False).mean()
                      DFexp3 = dfRM.ewm(span=9, adjust=False).mean()
                      DFMAVG288 = dfRM.rolling(window=288).mean()
                      DFMAVG48  = dfRM.rolling(window=48).mean()
                      DFMAVG6   = dfRM.rolling(window=6).mean()                     
                      DFMAVG200tech = dfRM.rolling(window=200).mean()
                      DFMAVG52tech  = dfRM.rolling(window=52).mean()
                      DFMAVG20tech  = dfRM.rolling(window=20).mean()
                      DFSTD200tech  = dfRM.rolling(window=200).std()
                      DFSTD52tech   = dfRM.rolling(window=52).std()
                      DFSTD20tech   = dfRM.rolling(window=20).std()
                      DFMACD = DFexp1-DFexp2
                      DFlastexp1row=DFexp1.tail(1)
                      DFlastexp2row=DFexp2.tail(1)
                      DFlastexp3row=DFexp3.tail(1)
                      DFlastRSI1row=DFRSI1.tail(1)
                      DFlastRSI2row=DFRSI2.tail(1)
                      DFlastMACDrow=DFMACD.tail(1)
                      DFlastMAVG288row =DFMAVG288.tail(1)
                      DFlastMAVG48row  =DFMAVG48.tail(1)
                      DFlastMAVG6row   =DFMAVG6.tail(1) 
                      DFlastMAVG200techrow =DFMAVG200tech.tail(1)
                      DFlastMAVG52techrow  =DFMAVG52tech.tail(1)
                      DFlastMAVG20techrow  =DFMAVG20tech.tail(1) 
                      DFlastSTD200techrow  =DFSTD200tech.tail(1)
                      DFlastSTD52techrow   =DFSTD52tech.tail(1)
                      DFlastSTD20techrow   =DFSTD20tech.tail(1) 
                      exp1    =float(DFlastexp1row['close'].values)
                      exp2    =float(DFlastexp2row['close'].values)
                      exp3    =float(DFlastexp3row['close'].values)
                      RSI1    =float(DFlastRSI1row['close'].values)
                      RSI2    =float(DFlastRSI2row['close'].values)
                      MACD    =float(DFlastMACDrow['close'].values)
                      MAVG288 =float(DFlastMAVG288row['close'].values)
                      MAVG48  =float(DFlastMAVG48row['close'].values)
                      MAVG6   =float(DFlastMAVG6row['close'].values)
                      MAVG200tech =float(DFlastMAVG200techrow['close'].values)
                      MAVG52tech  =float(DFlastMAVG52techrow['close'].values)
                      MAVG20tech  =float(DFlastMAVG20techrow['close'].values)
                      STD200tech  =float(DFlastSTD200techrow['close'].values)
                      STD52tech   =float(DFlastSTD52techrow['close'].values)
                      STD20tech   =float(DFlastSTD20techrow['close'].values)
                      #get last value dataframe MACD RSI

                      df=dfonedY
                      #print(df)
                      dfmax= pd.DataFrame()
                      dfmin= pd.DataFrame()
                      dfmax= df.rolling(16).max()
                      dfmax= dfmax.dropna(how='all', axis=0) 
                      dfmax= dfmax.reset_index()
                      dfmin= df.rolling(16).min()
                      dfmin= dfmin.dropna(how='all', axis=0) 
                      dfmin= dfmin.reset_index()

                      resupdf=dfmin
                      resupdf=resupdf.append(dfmax, ignore_index=True)
                      dfa=pd.DataFrame()

                      dfa['occur']=resupdf['close'].value_counts()
                      dfa['resitance']=dfa.index
                      resitancedf=dfa.head(14)
                      resitancedf.sort_index(inplace=True)
                      resitancedf.index = range(len(resitancedf))
                      snumber=len(resitancedf[-3:].query('resitance > '+str(close)))
                      rnumber=len(resitancedf[:3].query('resitance < '+str(close)))
                      if snumber==0:
                        s1=close;s2=close;s3=close
                      elif snumber==1:
                        s3=close;s2=close
                        s1=resitancedf.loc[resitancedf['resitance'] > close].iloc[0,1]
                      elif snumber==2: 
                        s3=close
                        s1=resitancedf.loc[resitancedf['resitance'] > close].iloc[0,1] 
                        s2=resitancedf.loc[resitancedf['resitance'] > close].iloc[1,1]
                      elif snumber==3:
                        s1=resitancedf.loc[resitancedf['resitance'] > close].iloc[0,1] 
                        s2=resitancedf.loc[resitancedf['resitance'] > close].iloc[1,1]   
                        s3=resitancedf.loc[resitancedf['resitance'] > close].iloc[2,1]
                      if rnumber==0:
                        r3=close;r2=close;r1=close
                      elif rnumber==1:
                        r1=close;r2=close
                        r3=resitancedf.loc[resitancedf['resitance'] < close].iloc[-1,1]
                      elif rnumber==2:
                        r1=close
                        r3=resitancedf.loc[resitancedf['resitance'] < close].iloc[-1,1]
                        r2=resitancedf.loc[resitancedf['resitance'] < close].iloc[-2,1]
                      elif rnumber==3:  
                        r3=resitancedf.loc[resitancedf['resitance'] < close].iloc[-1,1]
                        r2=resitancedf.loc[resitancedf['resitance'] < close].iloc[-2,1]                        
                        r1=resitancedf.loc[resitancedf['resitance'] < close].iloc[-3,1]

                    crosstype = "0"
                    cross16ma4   = maprice16 - maprice4
                    cross16ma156 = maprice16 - maprice156
                    cross4ma156  = maprice4  - maprice156
                    #only when sign changes !!
                    if np.sign(oldcross16ma4)  != np.sign(cross16ma4)  and oldcross16ma4   != 999:
                      crosstype = crosstype + str(np.sign(cross16ma4)) +   "crosss16mav4"
                    if np.sign(oldcross16ma156) != np.sign(cross16ma156) and oldcross16ma156  != 999:
                      crosstype = crosstype + str(np.sign(cross16ma156)) + "cros16mav156"
                    if np.sign(oldcross4ma156) != np.sign(cross4ma156) and oldcross4ma156  != 999:  
                      crosstype = crosstype + str(np.sign(cross4ma156)) +  "cross4mav156"
                    oldcross16ma4   = cross16ma4
                    oldcross16ma156 = cross16ma156
                    oldcross4ma156  = cross4ma156
                    
                    cross16std4   = stdprice16 - stdprice4
                    cross16std156 = stdprice16 - stdprice156
                    cross4std156  = stdprice4  - stdprice156
                    #only when sign changes !!
                    
                    if np.sign(oldcross16std4)  != np.sign(cross16std4)  and oldcross16std4   != 999:
                      crosstype = crosstype + str(np.sign(cross16std4))  +  "crosss16std4"
                    if np.sign(oldcross16std156) != np.sign(cross16std156) and oldcross16std156  != 999:
                      crosstype = crosstype + str(np.sign(cross16std156)) + "cros16std156"
                    if np.sign(oldcross4std156) != np.sign(cross4std156) and oldcross4std156  != 999:  
                      crosstype = crosstype + str(np.sign(cross4std156))  + "cross4std156"
                    
                    oldcross16std4   = cross16std4
                    oldcross16std156 = cross16std156
                    oldcross4std156  = cross4std156

                    if MACDhigh<np.abs(MACD):
                      MACDhigh=np.abs(MACD)
                    
                    if RSI1high<RSI1:
                      RSI1high=RSI1 
                      RSI2high=RSI2  
                      if exp2 > exp1 and exp2 > exp3:
                        if exp3 > exp2:
                          RSI1highOrder="GBR"
                        elif exp3 < exp2:
                          RSI1highOrder="GRB"
                        else:
                          RSI1highOrder="G"
                      if exp2 < exp1 and exp2 < exp3:    
                        if exp3 < exp2:
                          RSI1highOrder="RBG"
                        elif exp3 > exp2:
                          RSI1highOrder="BRG"
                        else:
                          RSI1highOrder="G"          
                    
                    if RSI2high<RSI2:
                      RSI2high=RSI2
                      RSI1high=RSI1
                      if exp2 > exp1 and exp2 > exp3:
                        if exp3 > exp2:
                          RSI2highOrder="GBR"
                        elif exp3 < exp2:
                          RSI2highOrder="GRB"
                        else:
                          RSI2highOrder="G"
                      if exp2 < exp1 and exp2 < exp3:    
                        if exp3 < exp2:
                          RSI2highOrder="RBG"
                        elif exp3 > exp2:
                          RSI2highOrder="BRG"
                        else:
                          RSI2highOrder="G"  

                    if  RSI1high > RSI2high :
                      RSIhighOrder = RSI1highOrder
                      RSIhigh=RSI1high
                    else:
                      RSIhighOrder = RSI2highOrder  
                      RSIhigh=RSI2high
                    if RSI1 > RSI2:
                      RSItrade = RSI1
                    else:
                      RSItrade = RSI2 


                    if RSI1high/10 > sensitivelvlRSI or RSI2high/10 > sensitivelvlRSI :
                      colorline= 'red'   # if there is a spike in the space between green and b/r line of MACD
                    if np.abs(MACD)> sensitivelvlMACD:
                      colorline= 'green'

                    #proccess trade chance
                    bollingerbandhigh=2
                    bollingerbandlow=2
                    previoustrade=previoustrade+1
                    if high>innerhigh:
                      innerhigh       = high
                    if low<innerlow:
                      innerlow        = low
                    deltaclose        = close-previousclose
                    if candlecountangle > 0 and deltaclose != 0 : 
                      angle             = math.degrees(math.atan(deltaclose/candlecountangle)*100)                    
                    
                    if ((maprice16+(stdprice16*bollingerbandhigh))<high and crosstype != "0" and previoustrade>delaytrade) or ((maprice16-(stdprice16*bollingerbandlow ))>low  and crosstype != "0" and previoustrade>delaytrade):
                      previoustrade=0 ;trade=close ;tradeprofit=high+0.1 ;tradecut=low-0.01 ;tradelength=32
                      MACDtrade = np.abs(MACD)
                      RSItradeOrder=RSIhighOrder
                      #tradeslicedf=tradeslicedf.append({'timestamp' : previoustradetime ,'close' : newline['close'],'angle' : angle,'delta' : deltaclose,'candlescount': candlecount}, ignore_index=True)
                      if len(dfonedY)>candlecountangle+1:
                        DFtriangle=dfonedY.tail(candlecountangle-1)
                        DFtriangle=DFtriangle.reset_index()
                        surfaceabove=0
                        for i,newline in DFtriangle.iterrows():
                          diff=((deltaclose/candlecountangle)*(i+1)+previousclose)-close
                          if diff<maxlowdistance:
                            maxlowdistance  = diff
                          if diff>maxhighdistance:
                            maxhighdistance = diff
                          surfaceabove=surfaceabove+diff
                      #diff              = (deltaclose/candletotalcount)
                      #surfaceabove      = surfaceabove+diff
                      #if diff<maxlowdistance:
                      #  maxlowdistance  = diff
                      #if diff>maxhighdistance:
                      #  maxhighdistance = diff
                      print(candlecountangle, " - ", colorline)
                      print("==================================")   
                      print("timestamp       : ",timestamp)
                      print("close           : ",close) 
                      print("angle           : ",angle) 
                      print("delta           : ",deltaclose) 
                      print("surfaceabove    ; ",surfaceabove)
                      print("innerhigh       ; ",innerhigh)
                      print("innerlow        ; ",innerlow)
                      print("maxhighdistance ; ",maxhighdistance)
                      print("maxlowdistance  ; ",maxlowdistance)

                    datestring,timestring = timestamp.split(" ") 
                    if timestring >= TimeSTARTTokyo and timestring < TimeENDTokyo:
                      OpenMarkets=OpenMarkets+"T"
                    if timestring >= TimeSTARTLondon and timestring < TimeENDLondon:
                      OpenMarkets=OpenMarkets+"L"
                    if timestring >= TimeSTARTNewYork and timestring < TimeENDNewYork:
                      OpenMarkets=OpenMarkets+"N"
                    
                    MAVGcross=""  
                    MVAGOrder=""
                    if MAVG288 > MAVG6 and MAVG288 > MAVG48:
                      if MAVG48 > MAVG6:
                        MVAGOrder=MVAGOrder+"GBR1"
                      else:
                        MVAGOrder=MVAGOrder+"GRB1"
                    if MAVG288 < MAVG6 and MAVG288 < MAVG48:
                      if MAVG48 < MAVG6:
                        MVAGOrder=MVAGOrder+"RBG1"
                      else:
                        MVAGOrder=MVAGOrder+"BRG1"
                    cross048mvg006 = MAVG48 - MAVG6
                    cross048mvg288 = MAVG48 - MAVG288
                    cross006mvg288 = MAVG6  - MAVG288
                    #only when sign changes !!
                    if np.sign(oldcross048mvg006)  != np.sign(cross048mvg006) and oldcross048mvg006  != 999:
                      MAVGcross = MAVGcross + str(np.sign(cross048mvg006)) + "cross048mvg006"
                    if np.sign(oldcross048mvg288) != np.sign(cross048mvg288)  and oldcross048mvg288  != 999:
                      MAVGcross = MAVGcross + str(np.sign(cross048mvg288)) + "cross048mvg288"
                    if np.sign(oldcross006mvg288) != np.sign(cross006mvg288)  and oldcross006mvg288  != 999:  
                      MAVGcross = MAVGcross + str(np.sign(cross006mvg288)) + "cross006mvg288"
                    oldcross048mvg006 = cross048mvg006
                    oldcross048mvg288 = cross048mvg288
                    oldcross006mvg288 = cross006mvg288

                    if MAVG200tech > MAVG20tech and MAVG200tech > MAVG52tech:
                      if MAVG52tech > MAVG20tech:
                        MVAGOrder=MVAGOrder+"GBR2"
                      else:
                        MVAGOrder=MVAGOrder+"GRB2"
                    if MAVG200tech < MAVG20tech and MAVG200tech < MAVG52tech:
                      if MAVG52tech < MAVG20tech:
                        MVAGOrder=MVAGOrder+"RBG2"
                      else:
                        MVAGOrder=MVAGOrder+"BRG2"
                    cross052mvg020 = MAVG52tech - MAVG20tech
                    cross052mvg200 = MAVG52tech - MAVG200tech
                    cross020mvg200 = MAVG20tech - MAVG200tech
                    #only when sign changes !!
                    if np.sign(oldcross052mvg020)  != np.sign(cross052mvg020) and oldcross052mvg020  != 999:
                      MAVGcross = MAVGcross + str(np.sign(cross052mvg020)) + "cross052mvg020"
                    if np.sign(oldcross052mvg200) != np.sign(cross052mvg200)  and oldcross052mvg200  != 999:
                      MAVGcross = MAVGcross + str(np.sign(cross052mvg200)) + "cross048mvg200"
                    if np.sign(oldcross020mvg200) != np.sign(cross020mvg200)  and oldcross020mvg200  != 999:  
                      MAVGcross = MAVGcross + str(np.sign(cross020mvg200)) + "cross006mvg200"
                    oldcross052mvg020 = cross052mvg020
                    oldcross052mvg200 = cross052mvg200
                    oldcross020mvg200 = cross020mvg200


                    # Start Process orders
                    fxpair = ticker
                    price  = close
                    amt    = 0.0
                    if len(orders) > 0:
                        fxpair,price,amt = str(orders[0]).split(",")
                        fxpair           = fxpair.replace("(","").replace("'","")
                        amt              = amt.replace(")","")
                        final_orders = [orders[k] for k in np.random.choice(len(orders), replace=False, size=len(orders))]
                        for order in final_orders:
                            portvalue=controller._portfolio.value_summary(timestamp)
                            controller.process_order(order,timestamp,portvalue)
                    portvalue = controller._portfolio.value_summary(timestamp)  
                    # End Process orders 
                    #add colorline!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
                    todf      = pd.DataFrame(columns=['timestamp', 'DateTime', 'OpenMarkets', 'ticker', 'shares', 'portvalue', 'trade', 'tradecolor','angle','RSItradeOrder','RSIhighOrder','RSIhigh','RSItrade','MACDhigh','MACDtrade','candletotalcount','innerhigh','innerlow','maxhighdistance','maxlowdistance','surfaceabove', 'tradeprofit', 'tradecut', 'tradelength', 'crosstype', 'min4', 'min156', 'min16', 'max4', 'max156', 'max16','ma4', 'std4', 'ma156', 'std156', 'ma16', 'std16', 'r3', 'r2', 'r1', 's1', 's2', 's3', 'open', 'close', 'high', 'low','exp1','exp2','exp3','MACD','RSI1','RSI2','MAVG288','MAVG48','MAVG6','MAVG200tech','MAVG52tech','MAVG20tech','STD200tech','STD52tech','STD20tech','MAVGcross','MVAGOrder'])
                    todf      = todf.append({'timestamp': timestamp, 'DateTime': DateTime, 'OpenMarkets': OpenMarkets, 'ticker': fxpair, 'shares': amt, 'portvalue': portvalue, 'trade': trade, 'tradecolor': colorline, 'angle':angle, 'RSItradeOrder':RSItradeOrder, 'RSIhighOrder':RSIhighOrder, 'RSIhigh':RSIhigh, 'RSItrade':RSItrade, 'MACDhigh':MACDhigh, 'MACDtrade':MACDtrade,'candletotalcount':candlecountangle,'innerhigh':innerhigh,'innerlow':innerlow,'maxhighdistance':maxhighdistance,'maxlowdistance':maxlowdistance,'surfaceabove':surfaceabove, 'tradeprofit': tradeprofit, 'tradecut': tradecut, 'tradelength': tradelength, 'crosstype': crosstype, 'min4': minprice4, 'min156': minprice156, 'min16': minprice16, 'max4': maxprice4, 'max156': maxprice156, 'max16': maxprice16, 'ma4': maprice4, 'std4': stdprice4,'ma156': maprice156, 'std156': stdprice156,'ma16': maprice16, 'std16': stdprice16, 'r3':r3, 'r2':r2, 'r1':r1, 's1':s1, 's2':s2, 's3':s3, 'open': open, 'close': close, 'high': high, 'low': low, 'exp1': exp1, 'exp2': exp2, 'exp3': exp3, 'MACD': MACD, 'RSI1': RSI1, 'RSI2': RSI2, 'MAVG288': MAVG288, 'MAVG48': MAVG48, 'MAVG6': MAVG6, 'MAVG200tech': MAVG200tech, 'MAVG52tech': MAVG52tech, 'MAVG20tech': MAVG20tech, 'STD200tech': STD200tech, 'STD52tech': STD52tech, 'STD20tech': STD20tech, 'MAVGcross': MAVGcross, 'MVAGOrder': MVAGOrder }, ignore_index=True)
                    todf.to_sql(databasename, con=engine, if_exists='append')
                    if trade != 0:
                      colorline= 'lightgrey'
                      RSI1high=RSI1
                      RSI2high=RSI2
                      MACDhigh=MACD
                      candlecountangle  = -1
                      previoustradetime = timestamp
                      previousclose     = close
                      surfaceabove      = -9999
                      innerhigh         = -9999;innerlow= 9999
                      maxhighdistance=-9999;maxlowdistance=9999
                    trade=0 
                    OpenMarkets=""


        except Exception as e:
            print(e)
        finally:
            print(controller._portfolio.value_summary(None))

    def process_order(self, order,timestamp,portvalue):
        success = False
        receipt = self._order_api.process_order(order)
        if receipt is not None:
            success = self.process_receipt(receipt,timestamp,portvalue)
        if order is None or success is False:
            print(('{order_type} failed: %s at $%s for %s shares' % order).format(order_type = 'Sell' if order[2] < 0 else 'Buy'))

    def process_receipt(self,receipt,timestamp,portvalue):
        ticker      = receipt[0]
        price       = receipt[1]
        share_delta = receipt[2]
        fee         = receipt[3]
        temp        = self._portfolio.balance - (price * share_delta + fee)
        if temp > 0:
            if share_delta < 0 and -share_delta > self._portfolio.get_shares(ticker):
                # Liquidate
                share_delta = -self._portfolio.get_shares(ticker)
                fee         = self._order_api._calculate_fee(share_delta*price)
                if fee > abs(share_delta*price):
                    return False
            self._portfolio.update_trade(ticker=ticker, price=price, share_delta=share_delta, fee=fee)
            return True
        return False

    def process_pricing(self, ticker, price, timestamp):
        self._portfolio.update(price=price, ticker  = ticker, timestamp = timestamp)
        self._algorithm.update(stock=ticker, price  = price, timestamp = timestamp)
        self._algorithm.update1(stock=ticker, price = price, timestamp = timestamp)
        self._algorithm.update2(stock=ticker, price = price, timestamp = timestamp)
        self._algorithm.update10(stock=ticker, price = price, timestamp = timestamp)


class Backtester:

    def backtest(self):
       
        # Initiate run
        q  = Queue()
        ds = None
        c  = None

        ds = DataSource(
            source= 'yahoo',
            start= dt.datetime(2018,1,1),
            end=dt.datetime(2018,12,31),
            tickers= ['USDJPY']
        )
        c = Controller(
            portfolio=Portfolio(),
            algorithm=Algorithm()
        )

        p  = Process(target=DataSource.process, args=((q,ds)))
        p1 = Process(target=Controller.backtest, args=((q,c)))

        p.start()
        p1.start()
        p.join()
        p1.join()

if __name__ == '__main__':
    b = Backtester()
    b.backtest()
!ls -sla "database/"
!cp   database/TradeOrder.db  "/gdrive/My Drive/5MFXDB/TradeOrder2018refactor1.db"
#!cp   database/TradeOrder.db  database/TradeOrder201875.db 
#!rm database/TradeOrder.db
!ls -sla "/gdrive/My Drive/5MFXDB/"